# 🛡️ NetGuardian-AI: Complete IDS Pipeline (Part 3)

**Phases 6-7: Real-Time Simulation & Model Comparison**

This final part covers:
- Phase 6: Real-Time Simulation
- Phase 7: Model Comparison Benchmark

---

<a id='phase6'></a>
## 🚨 Phase 6: Real-Time Simulation

**Purpose**: Simulate real-world deployment of the IDS.

**What this phase does**:
- Simulates packet-by-packet processing
- Generates real-time alerts for detected attacks
- Demonstrates the system's operational behavior
- Provides statistics on detection performance

### Step 6.1: Prepare Simulation Data

**Explanation**: We create a stream of network packets (mix of normal and malicious) to simulate real-time traffic.

In [ ]:
from IPython.display import display, clear_output
import time

# Sample random packets for simulation
stream_data = df_clean.sample(n=50, random_state=42).reset_index(drop=True)

# Prepare features (exclude labels)
label_cols = ['Attack_Type', 'Binary_Label', 'Attack_Merged', 'Multiclass_Label']
features_to_predict = stream_data.drop(label_cols, axis=1, errors='ignore')

print(f"✅ Prepared {len(stream_data)} packets for simulation")
print(f"\nPacket distribution:")
print(stream_data['Attack_Type'].value_counts())

### Step 6.2: Run Real-Time Simulation

**Explanation**: This simulates the IDS processing packets one by one:
- **Green alerts**: Normal traffic (no action needed)
- **Red alerts**: Attack detected (security team notified)
- **Confidence score**: How certain the model is

In production, this would run continuously on live network traffic.

In [ ]:
print("🟢 Starting NetGuardian-AI Real-Time Monitor...")
print("="*60)

history = []
attack_count = 0
normal_count = 0

for i in range(len(stream_data)):
    # Extract one packet
    packet_features = features_to_predict.iloc[i:i+1]
    true_label = stream_data.iloc[i]['Attack_Type']
    
    # Predict
    prediction = hybrid_ids.predict(packet_features)[0]
    
    # Display alert
    if prediction['is_attack']:
        attack_count += 1
        print(f"\033[91m🚨 [PACKET {i:03d}] ATTACK DETECTED: {prediction['type']} "
              f"(Confidence: {prediction['confidence']:.2%}) | True: {true_label}\033[0m")
    else:
        normal_count += 1
        if i % 5 == 0:  # Print every 5th normal packet to reduce clutter
            print(f"\033[92m✅ [PACKET {i:03d}] Normal Traffic | True: {true_label}\033[0m")
    
    # Store for statistics
    history.append({
        'packet_id': i,
        'predicted': prediction['type'],
        'true': true_label,
        'confidence': prediction['confidence'],
        'is_attack': prediction['is_attack']
    })
    
    # Simulate real-time delay
    time.sleep(0.05)

print("="*60)
print("🏁 Simulation Complete")
print(f"\nSummary:")
print(f"  Total packets processed: {len(stream_data)}")
print(f"  Normal traffic: {normal_count}")
print(f"  Attacks detected: {attack_count}")

### Step 6.3: Session Statistics

**Explanation**: Analyze the simulation results to understand detection performance.

In [ ]:
# Convert to DataFrame for analysis
history_df = pd.DataFrame(history)

# Detection statistics
print("📊 Detection Statistics:")
print("\nPredicted attack types:")
print(history_df['predicted'].value_counts())

# Accuracy
correct = (history_df['predicted'] == history_df['true']).sum()
accuracy = correct / len(history_df) * 100

print(f"\n✅ Accuracy: {accuracy:.2f}% ({correct}/{len(history_df)} correct)")

# Average confidence
avg_confidence = history_df['confidence'].mean()
print(f"📈 Average Confidence: {avg_confidence:.2%}")

<a id='phase7'></a>
## 🏆 Phase 7: Model Comparison Benchmark

**Purpose**: Compare different ML approaches for IDS.

**Models Tested**:
1. **Random Forest**: Ensemble of decision trees
2. **SVM**: Support Vector Machine with RBF kernel
3. **KNN**: K-Nearest Neighbors
4. **Autoencoder**: Deep learning anomaly detection

**Why Compare?**
- Understand trade-offs (speed vs accuracy)
- Validate our XGBoost choice
- Explore unsupervised learning (Autoencoder)

### Step 7.1: Prepare Benchmark Dataset

**Explanation**: We use a smaller sample (50k) because SVM and KNN are computationally expensive on large datasets.

In [ ]:
# Sample for benchmark (stratified)
df_benchmark, _ = train_test_split(
    df_clean, 
    train_size=50000, 
    stratify=df_clean['Binary_Label'], 
    random_state=42
)

# Prepare features
X_bench = df_benchmark.drop(label_cols, axis=1, errors='ignore')
y_bench = df_benchmark['Binary_Label']

# Split
X_train_bench, X_test_bench, y_train_bench, y_test_bench = train_test_split(
    X_bench, y_bench, test_size=0.3, random_state=42, stratify=y_bench
)

# Scale
scaler_bench = StandardScaler()
X_train_bench_scaled = scaler_bench.fit_transform(X_train_bench)
X_test_bench_scaled = scaler_bench.transform(X_test_bench)

print(f"✅ Benchmark dataset prepared")
print(f"Training: {X_train_bench_scaled.shape}")
print(f"Testing: {X_test_bench_scaled.shape}")

### Step 7.2: Train and Evaluate Supervised Models

**Explanation**: We train 3 supervised learning algorithms and compare their performance.

In [ ]:
# Results storage
results = {}

def train_evaluate(model, name):
    """Train and evaluate a model"""
    print(f"\n🔄 Training {name}...")
    start = time.time()
    model.fit(X_train_bench_scaled, y_train_bench)
    train_time = time.time() - start
    
    # Predict
    y_pred = model.predict(X_test_bench_scaled)
    
    # Metrics
    acc = accuracy_score(y_test_bench, y_pred)
    f1 = f1_score(y_test_bench, y_pred)
    prec = precision_score(y_test_bench, y_pred)
    rec = recall_score(y_test_bench, y_pred)
    
    # Save model
    filename = f"{name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, filename)
    
    results[name] = {
        'Time_sec': train_time,
        'Accuracy': acc,
        'F1-Score': f1,
        'Precision': prec,
        'Recall': rec,
        'File': filename
    }
    
    print(f"✅ {name} completed in {train_time:.2f}s | F1: {f1:.4f}")

# Train models
print("="*70)
print("SUPERVISED LEARNING BENCHMARK")
print("="*70)

# 1. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
train_evaluate(rf, 'Random Forest')

# 2. SVM
svm = SVC(kernel='rbf', random_state=42)
train_evaluate(svm, 'SVM')

# 3. KNN
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
train_evaluate(knn, 'KNN')

### Step 7.3: Train Autoencoder (Unsupervised)

**Explanation**: 
- **Autoencoder** learns to compress and reconstruct NORMAL traffic
- High reconstruction error → Anomaly (Attack)
- This is **unsupervised**: doesn't need attack labels for training
- Useful for detecting **zero-day attacks** (never seen before)

In [ ]:
print("\n" + "="*70)
print("UNSUPERVISED LEARNING: AUTOENCODER")
print("="*70)

# Train only on normal traffic
X_train_normal = X_train_bench_scaled[y_train_bench == 0]
X_test_normal = X_test_bench_scaled[y_test_bench == 0]

print(f"Training on {len(X_train_normal):,} normal samples")

# Build autoencoder
input_dim = X_train_bench_scaled.shape[1]

autoencoder = Sequential([
    # Encoder
    Dense(32, activation='relu', input_shape=(input_dim,)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),  # Bottleneck (compressed representation)
    # Decoder
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')  # Reconstruction
])

autoencoder.compile(optimizer='adam', loss='mse')

# Train
print("\n🔄 Training Autoencoder...")
start = time.time()
history = autoencoder.fit(
    X_train_normal, X_train_normal,
    epochs=20,
    batch_size=64,
    validation_data=(X_test_normal, X_test_normal),
    shuffle=True,
    verbose=0
)
train_time = time.time() - start
print(f"✅ Training completed in {train_time:.2f}s")

In [ ]:
# Calculate reconstruction error
reconstructions = autoencoder.predict(X_test_bench_scaled, verbose=0)
mse = np.mean(np.power(X_test_bench_scaled - reconstructions, 2), axis=1)

# Set threshold (95th percentile of normal traffic error)
normal_mse = mse[y_test_bench == 0]
threshold = np.percentile(normal_mse, 95)
print(f"\nAnomaly detection threshold: {threshold:.4f}")

# Predict: High error = Attack
y_pred_ae = (mse > threshold).astype(int)

# Evaluate
results['Autoencoder'] = {
    'Time_sec': train_time,
    'Accuracy': accuracy_score(y_test_bench, y_pred_ae),
    'F1-Score': f1_score(y_test_bench, y_pred_ae),
    'Precision': precision_score(y_test_bench, y_pred_ae),
    'Recall': recall_score(y_test_bench, y_pred_ae),
    'File': 'autoencoder.h5'
}

# Save
autoencoder.save('autoencoder.h5')
print(f"✅ Autoencoder saved | F1: {results['Autoencoder']['F1-Score']:.4f}")

### Step 7.4: Compare All Models

**Explanation**: We visualize the trade-offs between different approaches.

In [ ]:
# Create comparison DataFrame
df_results = pd.DataFrame(results).T
df_results = df_results.sort_values(by='F1-Score', ascending=False)

print("\n" + "="*70)
print("FINAL BENCHMARK RESULTS")
print("="*70)
display(df_results)
print("="*70)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# F1-Score comparison
df_results['F1-Score'].plot(kind='bar', ax=axes[0], color='skyblue', edgecolor='black')
axes[0].set_title('Performance Comparison (F1-Score)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('F1-Score')
axes[0].set_ylim(0, 1.1)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Training time comparison
df_results['Time_sec'].plot(kind='bar', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Time (seconds)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### Step 7.5: Key Insights

**Interpretation Guide**:

**Performance (F1-Score)**:
- **Random Forest**: Usually best balance of speed and accuracy
- **SVM**: High accuracy but slow on large datasets
- **KNN**: Fast prediction but memory-intensive
- **Autoencoder**: Good for unknown attacks but may have more false positives

**Training Time**:
- **KNN**: Fastest (no training, just stores data)
- **Random Forest**: Moderate
- **SVM**: Slowest (doesn't scale well)
- **Autoencoder**: Depends on epochs and architecture

**When to Use Each**:
- **Random Forest/XGBoost**: Production IDS (best overall)
- **SVM**: Small, critical systems where accuracy is paramount
- **KNN**: Quick prototyping, small datasets
- **Autoencoder**: Detecting novel/zero-day attacks

## 🎓 Summary & Next Steps

### What We Accomplished

✅ **Phase 0**: Environment setup and library installation

✅ **Phase 1**: Dataset construction and cleaning
- Loaded CICIDS2017 dataset
- Fixed data quality issues (NaN, infinites, duplicates)
- Cleaned and standardized features

✅ **Phase 2**: MITRE ATT&CK analysis
- Mapped attacks to cybersecurity framework
- Analyzed network characteristics

✅ **Phase 3**: Data preparation
- Created binary and multi-class labels
- Normalized features
- Split train/test sets

✅ **Phase 4**: Hybrid model training
- Trained binary detection model (XGBoost)
- Trained multi-class classifier with SMOTE
- Created two-stage hybrid system

✅ **Phase 5**: Model evaluation
- Tested on clean data
- Robustness testing with noise
- Error analysis

✅ **Phase 6**: Real-time simulation
- Simulated packet-by-packet processing
- Generated real-time alerts

✅ **Phase 7**: Model comparison
- Benchmarked 4 different approaches
- Compared supervised vs unsupervised learning

### Next Steps for Production

1. **Deploy to Cloud**: Use the saved models in a production environment
2. **API Development**: Create REST API for real-time predictions
3. **Dashboard**: Build monitoring dashboard (Grafana/Kibana)
4. **Continuous Learning**: Implement model retraining pipeline
5. **Alert System**: Integrate with SIEM (Security Information and Event Management)
6. **Performance Optimization**: Use model quantization for faster inference

### Files Generated

```
models/
├── model1_binary.pkl           # Binary detection model
├── model2_multiclass.pkl       # Multi-class classifier
├── hybrid_ids_system.pkl       # Complete hybrid system
├── scaler.pkl                  # Feature scaler
├── label_encoder.pkl           # Label encoder
├── random_forest.pkl           # Benchmark: Random Forest
├── svm.pkl                     # Benchmark: SVM
├── knn.pkl                     # Benchmark: KNN
└── autoencoder.h5              # Benchmark: Autoencoder
```

### Resources

- **CICIDS2017 Dataset**: https://www.kaggle.com/datasets/cicdataset/cicids2017
- **MITRE ATT&CK**: https://attack.mitre.org/
- **XGBoost Documentation**: https://xgboost.readthedocs.io/
- **SMOTE Paper**: https://arxiv.org/abs/1106.1813

---

**🎉 Congratulations! You've built a complete hybrid IDS system! 🎉**